In [21]:
import pandas as pd
from pathlib import Path
from typing import Union

def simple_csv_concat(base_path: Union[str, Path]) -> pd.DataFrame:
    """
    Simple function to read and concatenate all CSV files from folders.
    Automatically handles date formatting for common date columns.
    
    Args:
        base_path: Root directory containing folders with CSV files
    
    Returns:
        pd.DataFrame: Concatenated DataFrame with properly formatted dates
    """
    base_path = Path(base_path)
    csv_files = list(base_path.rglob("*.csv"))
    
    if not csv_files:
        print("No CSV files found.")
        return pd.DataFrame()
    
    print(f"Found {len(csv_files)} CSV files to process...")
    
    dataframes = []
    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)
            
            # Clean column names (remove trailing spaces)
            df.columns = df.columns.str.strip()
            
            # Format date columns
            df = format_date_columns(df)
            
            dataframes.append(df)
            
        except Exception as e:
            print(f"✗ Error loading {csv_file}: {e}")
    
    if not dataframes:
        return pd.DataFrame()
    
    result_df = pd.concat(dataframes, ignore_index=True)
    result_df = result_df.drop(columns=['Shares Traded','Turnover (₹ Cr)'])
    print(f"✓ Final DataFrame: {len(result_df)} rows × {len(result_df.columns)} columns")
    
    return result_df


def format_date_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Format date columns in the DataFrame.
    Handles common date column names and formats.
    
    Args:
        df: Input DataFrame
    
    Returns:
        DataFrame with properly formatted date columns
    """
    # Common date column names to check
    date_columns = ['Date', 'date', 'DATE', 'Date ', 'Timestamp', 'timestamp']
    
    for col in df.columns:
        if col in date_columns:
            try:
                # Convert to datetime with common formats
                df[col] = pd.to_datetime(df[col], format='%d-%b-%Y', errors='coerce')
                
                # If that fails, try automatic parsing
                if df[col].isna().all():
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                
            except Exception as e:
                print(f"  ✗ Could not format date column {col}: {e}")
    
    return df


In [22]:
df = simple_csv_concat(base_path="/Users/mayurgd/Documents/CodingSpace/market_analysis/datasets/daily-dataset")

Found 13 CSV files to process...
✓ Final DataFrame: 3056 rows × 5 columns


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Tuple, Optional

class BTSTBreakoutBacktester:
    """
    BTST (Buy Today Sell Tomorrow) Backtesting class for breakout strategy:
    - Moving window of 5 days
    - If 4th candle breaks high/low of first 3 candles
    - Enter trade at 4th candle's close
    - Exit at next day's (5th candle) open
    - Measure points gained/lost in direction of breakout
    - Fixed position size of 150 quantities per trade
    """
    
    def __init__(self, data: List[Dict], initial_capital: float = 10000000):
        """
        Initialize backtester with data and capital.
        
        Args:
            data: List of dictionaries with OHLC data
            initial_capital: Starting capital
        """
        self.df = pd.DataFrame(data)
        self.initial_capital = initial_capital
        self.capital = initial_capital
        self.trades = []
        
        # Ensure Date is datetime
        if 'Date' in self.df.columns:
            self.df['Date'] = pd.to_datetime(self.df['Date'])
        
        # Sort by date
        self.df = self.df.sort_values('Date').reset_index(drop=True)
        
    def identify_breakouts(self) -> pd.DataFrame:
        """
        Identify breakout opportunities using 5-day moving window.
        
        Returns:
            DataFrame with breakout signals and trade details
        """
        df = self.df.copy()
        
        # Initialize columns for signals and trade info
        df['signal'] = 0
        df['signal_type'] = 'NONE'
        df['reference_high'] = np.nan
        df['reference_low'] = np.nan
        df['entry_price'] = np.nan
        df['exit_price'] = np.nan
        df['points_gained'] = np.nan
        
        # Process each 5-day window
        for i in range(3, len(df) - 1):  # Start from index 3 (4th candle), leave room for exit
            # Get the 5-day window
            window_start = i - 3  # First candle index
            window_end = i        # Fourth candle index (current)
            exit_index = i + 1    # Fifth candle index (next day)
            
            # First 3 candles (days 1, 2, 3)
            first_three = df.iloc[window_start:window_start + 3]
            reference_high = first_three['High'].max()
            reference_low = first_three['Low'].min()
            
            # Fourth candle (current day)
            current_candle = df.iloc[i]
            
            # Fifth candle (next day for exit)
            next_candle = df.iloc[exit_index]
            
            # Store reference levels
            df.at[i, 'reference_high'] = reference_high
            df.at[i, 'reference_low'] = reference_low
            
            # Check for breakout conditions on 4th candle
            breakout_high = current_candle['Close'] > reference_high
            breakout_low = current_candle['Close'] < reference_low
            
            if breakout_high:
                # Bullish breakout - expect higher opening next day
                df.at[i, 'signal'] = 1
                df.at[i, 'signal_type'] = 'BTST_LONG'
                df.at[i, 'entry_price'] = current_candle['Close']
                df.at[i, 'exit_price'] = next_candle['Open']
                df.at[i, 'points_gained'] = next_candle['Open'] - current_candle['Close']
                
            elif breakout_low:
                # Bearish breakdown - expect lower opening next day
                df.at[i, 'signal'] = -1
                df.at[i, 'signal_type'] = 'BTST_SHORT'
                df.at[i, 'entry_price'] = current_candle['Close']
                df.at[i, 'exit_price'] = next_candle['Open']
                df.at[i, 'points_gained'] = current_candle['Close'] - next_candle['Open']  # For short, gain when price falls
        
        return df
    
    def backtest(self, quantity: int = 150) -> Dict:
        """
        Run the BTST backtest with fixed position size.
        
        Args:
            quantity: Fixed number of shares/contracts per trade (default: 150)
            
        Returns:
            Dictionary with backtest results
        """
        # Reset for new backtest
        self.capital = self.initial_capital
        self.trades = []
        
        # Identify all breakout opportunities
        df = self.identify_breakouts()
        
        # Execute trades
        for i, row in df.iterrows():
            if row['signal'] != 0 and not pd.isna(row['entry_price']):
                # Fixed position size
                trade_quantity = quantity
                trade_capital = trade_quantity * row['entry_price']
                
                # Check if we have enough capital for the trade
                if trade_capital > self.capital:
                    # Skip trade if insufficient capital
                    continue
                
                if row['signal_type'] == 'BTST_LONG':
                    # Long trade: Buy at close, sell at next open
                    gross_pnl = row['points_gained'] * 0.7 *trade_quantity
                    
                elif row['signal_type'] == 'BTST_SHORT':
                    # Short trade: Sell at close, cover at next open
                    gross_pnl = row['points_gained'] * 0.7 * trade_quantity
                
                # Calculate net P&L (assuming minimal brokerage for simplicity)
                brokerage = 150  # 0.05% total brokerage
                net_pnl = gross_pnl - brokerage
                
                # Update capital
                self.capital += net_pnl
                
                # Record trade
                trade = {
                    'trade_date': row['Date'],
                    'exit_date': df.iloc[i + 1]['Date'] if i + 1 < len(df) else None,
                    'signal_type': row['signal_type'],
                    'reference_high': row['reference_high'],
                    'reference_low': row['reference_low'],
                    'entry_price': row['entry_price'],
                    'exit_price': row['exit_price'],
                    'quantity': trade_quantity,
                    'trade_capital': trade_capital,
                    'points_gained': row['points_gained'],
                    'gross_pnl': gross_pnl,
                    'brokerage': brokerage,
                    'net_pnl': net_pnl,
                    'return_pct': (net_pnl / trade_capital) * 100,
                    'capital_after_trade': self.capital
                }
                
                self.trades.append(trade)
        
        return self._generate_results()
    
    def _generate_results(self) -> Dict:
        """Generate comprehensive backtest results."""
        if not self.trades:
            return {
                'total_return': 0,
                'total_trades': 0,
                'long_trades': 0,
                'short_trades': 0,
                'winning_trades': 0,
                'losing_trades': 0,
                'win_rate': 0,
                'long_win_rate': 0,
                'short_win_rate': 0,
                'avg_points_per_trade': 0,
                'avg_return_per_trade': 0,
                'total_points_gained': 0,
                'total_brokerage': 0,
                'best_trade': None,
                'worst_trade': None,
                'final_capital': self.capital,
                'avg_capital_per_trade': 0,
                'max_capital_used': 0,
                'trades': []
            }
        
        trades_df = pd.DataFrame(self.trades)
        
        # Calculate metrics
        total_return = (self.capital - self.initial_capital) / self.initial_capital * 100
        win_rate = (trades_df['net_pnl'] > 0).mean() * 100
        avg_points_per_trade = trades_df['points_gained'].mean()
        avg_return_per_trade = trades_df['return_pct'].mean()
        
        # Separate long and short trades
        long_trades = trades_df[trades_df['signal_type'] == 'BTST_LONG']
        short_trades = trades_df[trades_df['signal_type'] == 'BTST_SHORT']
        
        return {
            'total_return': total_return,
            'total_trades': len(self.trades),
            'long_trades': len(long_trades),
            'short_trades': len(short_trades),
            'winning_trades': len(trades_df[trades_df['net_pnl'] > 0]),
            'losing_trades': len(trades_df[trades_df['net_pnl'] <= 0]),
            'win_rate': win_rate,
            'avg_points_per_trade': avg_points_per_trade,
            'avg_return_per_trade': avg_return_per_trade,
            'total_points_gained': trades_df['points_gained'].sum(),
            'total_brokerage': trades_df['brokerage'].sum(),
            'best_trade': trades_df.loc[trades_df['net_pnl'].idxmax()].to_dict() if not trades_df.empty else None,
            'worst_trade': trades_df.loc[trades_df['net_pnl'].idxmin()].to_dict() if not trades_df.empty else None,
            'long_win_rate': (long_trades['net_pnl'] > 0).mean() * 100 if len(long_trades) > 0 else 0,
            'short_win_rate': (short_trades['net_pnl'] > 0).mean() * 100 if len(short_trades) > 0 else 0,
            'final_capital': self.capital,
            'avg_capital_per_trade': trades_df['trade_capital'].mean(),
            'max_capital_used': trades_df['trade_capital'].max(),
            'trades': self.trades
        }
    
    def print_results(self, results: Dict):
        """Print formatted backtest results."""
        print("=" * 70)
        print("BTST BREAKOUT STRATEGY BACKTEST RESULTS")
        print("Fixed Position Size: 150 quantities per trade")
        print("=" * 70)
        print(f"Initial Capital: ₹{self.initial_capital:,.2f}")
        print(f"Final Capital: ₹{results['final_capital']:,.2f}")
        print(f"Total Return: {results['total_return']:.2f}%")
        print(f"Total P&L: ₹{results['final_capital'] - self.initial_capital:,.2f}")
        print()
        print("TRADE STATISTICS:")
        print(f"Total Trades: {results['total_trades']}")
        print(f"Long Trades: {results['long_trades']}")
        print(f"Short Trades: {results['short_trades']}")
        print(f"Winning Trades: {results['winning_trades']}")
        print(f"Losing Trades: {results['losing_trades']}")
        print(f"Overall Win Rate: {results['win_rate']:.2f}%")
        print(f"Long Win Rate: {results['long_win_rate']:.2f}%")
        print(f"Short Win Rate: {results['short_win_rate']:.2f}%")
        print()
        print("PERFORMANCE METRICS:")
        print(f"Average Points per Trade: {results['avg_points_per_trade']:.2f}")
        print(f"Total Points Gained: {results['total_points_gained']:.2f}")
        print(f"Average Return per Trade: {results['avg_return_per_trade']:.2f}%")
        print(f"Total Brokerage Paid: ₹{results['total_brokerage']:.2f}")
        print(f"Average Capital per Trade: ₹{results['avg_capital_per_trade']:,.2f}")
        print(f"Maximum Capital Used: ₹{results['max_capital_used']:,.2f}")
        print()
        
        if results['best_trade']:
            print("BEST TRADE:")
            best = results['best_trade']
            print(f"  Date: {best['trade_date'].strftime('%Y-%m-%d')} | Type: {best['signal_type']}")
            print(f"  Entry: {best['entry_price']:.2f} | Exit: {best['exit_price']:.2f}")
            print(f"  Points Gained: {best['points_gained']:.2f} | P&L: ₹{best['net_pnl']:.2f}")
            print(f"  Capital Used: ₹{best['trade_capital']:,.2f}")
        
        if results['worst_trade']:
            print("WORST TRADE:")
            worst = results['worst_trade']
            print(f"  Date: {worst['trade_date'].strftime('%Y-%m-%d')} | Type: {worst['signal_type']}")
            print(f"  Entry: {worst['entry_price']:.2f} | Exit: {worst['exit_price']:.2f}")
            print(f"  Points Lost: {worst['points_gained']:.2f} | P&L: ₹{worst['net_pnl']:.2f}")
            print(f"  Capital Used: ₹{worst['trade_capital']:,.2f}")
        
        print("=" * 70)
    
    def print_trade_details(self):
        """Print details of all trades."""
        if not self.trades:
            print("No trades executed.")
            return
        
        print("\nDETAILED TRADE LOG:")
        print("-" * 130)
        print(f"{'Date':<12} {'Type':<12} {'Entry':<8} {'Exit':<8} {'Points':<8} {'Capital':<10} {'P&L':<10} {'Return%':<8} {'Balance':<12}")
        print("-" * 130)
        
        for trade in self.trades:
            print(f"{trade['trade_date'].strftime('%Y-%m-%d'):<12} "
                  f"{trade['signal_type']:<12} "
                  f"{trade['entry_price']:<8.2f} "
                  f"{trade['exit_price']:<8.2f} "
                  f"{trade['points_gained']:<8.2f} "
                  f"₹{trade['trade_capital']:<9,.0f} "
                  f"₹{trade['net_pnl']:<9.2f} "
                  f"{trade['return_pct']:<7.2f}% "
                  f"₹{trade['capital_after_trade']:<11,.0f}")

In [47]:
# Initialize with your OHLC data
backtester = BTSTBreakoutBacktester(df)

# Run backtest with fixed 150 quantity (default)
results = backtester.backtest()

# Print results
backtester.print_results(results)
backtester.print_trade_details()

BTST BREAKOUT STRATEGY BACKTEST RESULTS
Fixed Position Size: 150 quantities per trade
Initial Capital: ₹10,000,000.00
Final Capital: ₹12,960,772.00
Total Return: 29.61%
Total P&L: ₹2,960,772.00

TRADE STATISTICS:
Total Trades: 1245
Long Trades: 738
Short Trades: 507
Winning Trades: 719
Losing Trades: 526
Overall Win Rate: 57.75%
Long Win Rate: 70.87%
Short Win Rate: 38.66%

PERFORMANCE METRICS:
Average Points per Trade: 12.04
Total Points Gained: 14988.20
Average Return per Trade: 0.05%
Total Brokerage Paid: ₹186750.00
Average Capital per Trade: ₹3,833,997.58
Maximum Capital Used: ₹7,864,815.00

BEST TRADE:
  Date: 2025-04-04 | Type: BTST_SHORT
  Entry: 22904.45 | Exit: 21758.40
  Points Gained: 1146.05 | P&L: ₹240520.50
  Capital Used: ₹6,871,335.00
WORST TRADE:
  Date: 2016-11-08 | Type: BTST_LONG
  Entry: 8543.55 | Exit: 8067.50
  Points Lost: -476.05 | P&L: ₹-100120.50
  Capital Used: ₹2,563,065.00

DETAILED TRADE LOG:
---------------------------------------------------------------